In [ ]:
import pandas as pd
import re

# Function to clean locations
def clean_location(location):
    # Remove leading and trailing whitespaces
    location = location.strip()

    # Remove special characters
    location = re.sub(r'[^a-zA-Z\s]', '', location)

    # Remove numbers
    location = re.sub(r'\d', '', location)

    return location

# Read Excel file
file_path = 'locations.xlsx'
df = pd.read_excel(file_path, header=None, names=['Location'])

# Clean locations column
df['Location'] = df['Location'].apply(clean_location)

# Remove rows where 'et al.' exists alone or as part of a phrase
df = df[~df['Location'].str.contains(r'\b(et al\.|et al)\b', case=False, regex=True)]

# Remove rows where the cleaned 'Location' starts with 'doi'
df = df[~df['Location'].str.lower().str.startswith('doi')]

# Remove duplicate rows
df.drop_duplicates(subset=['Location'], keep='first', inplace=True)

# Save cleaned data to a new Excel file
output_file_path = 'locations_cleaned.xlsx'
df.to_excel(output_file_path, index=False, header=True)

print(f'Data cleaned and saved to {output_file_path}')


In [ ]:
# Read Excel file
file_path = 'locations.xlsx'
df = pd.read_excel(file_path, header=None, names=['Location'])

In [ ]:
df['Location'] = df['Location'].str.lower()
df.drop_duplicates(subset=['Location'], keep='first', inplace=True)


In [ ]:
# Save cleaned data to a new Excel file
output_file_path = 'locations_cleaned.xlsx'
df.to_excel(output_file_path, index=False, header=True)

In [ ]:
pip install fuzzywuzzy

In [ ]:
import pandas as pd

# Read cleaned locations file
locations_df = pd.read_excel('locations_cleaned.xlsx', header=None, names=['Location'])

# List of other Excel files
keyword_files = ['Keyphrases_aws.xlsx', 'Keyphrases_tf-idf.xlsx', 'Keyphrases_yake.xlsx']

# Dictionary to store findings
findings = {}

# Iterate through each keyword file
for keyword_file in keyword_files:
    # Read keyword file
    keyword_df = pd.read_excel(keyword_file)

    # Convert columns to lowercase for case-insensitive comparison
    keyword_df['Keyword'] = keyword_df['Keyword'].str.lower()
    locations_df['Location'] = locations_df['Location'].str.lower()

    # Merge the cleaned locations with the keywords on 'Location'
    merged_df = pd.merge(keyword_df, locations_df, how='inner', left_on='Keyword', right_on='Location')

    # Extract matched locations
    matched_locations = merged_df['Location'].tolist()

    # Calculate the percentage of locations in each keyword file
    percentage = (len(matched_locations) / len(keyword_df)) * 100

    # Store findings in the dictionary
    findings[keyword_file] = {
        'Total Keywords': len(keyword_df),
        'Locations Matched': len(matched_locations),
        'Percentage Match': percentage,
        'Matched Locations': matched_locations
    }

# Print the findings
for keyword_file, result in findings.items():
    print(f'Findings for {keyword_file}:')
    print(f'Total Keywords: {result["Total Keywords"]}')
    print(f'Locations Matched: {result["Locations Matched"]}')
    print(f'Percentage Match: {result["Percentage Match"]:.2f}%')
    print(f'Matched Locations: {result["Matched Locations"]}\n')

In [ ]:
#fuzzy all
import pandas as pd
from fuzzywuzzy import process

# Read cleaned locations file
locations_df = pd.read_excel('locations_cleaned.xlsx', header=None, names=['Location'])

# List of other Excel files
keyword_files = ['Keyphrases_aws.xlsx', 'Keyphrases_tf-idf.xlsx', 'Keyphrases_yake.xlsx']

# Dictionary to store findings
findings = {}

# Fuzzy matching threshold (adjust as needed)
threshold = 95

# Iterate through each keyword file
for keyword_file in keyword_files:
    # Read keyword file
    keyword_df = pd.read_excel(keyword_file)

    # Convert columns to lowercase for case-insensitive comparison
    keyword_df['Keyword'] = keyword_df['Keyword'].str.lower()
    locations_df['Location'] = locations_df['Location'].str.lower()

    # Dictionary to store matched locations
    matched_locations_dict = {}

    # Iterate through each keyword
    for keyword in keyword_df['Keyword']:
        # Use fuzzywuzzy process to find the best match
        match, score, index = process.extractOne(keyword, locations_df['Location'])
        
        # Check if the match meets the threshold
        if score >= threshold:
            # Store the matched location
            matched_locations_dict[keyword] = match

    # Calculate the percentage of locations in each keyword file
    percentage = (len(matched_locations_dict) / len(keyword_df)) * 100

    # Store findings in the dictionary
    findings[keyword_file] = {
        'Total Keywords': len(keyword_df),
        'Locations Matched': len(matched_locations_dict),
        'Percentage Match': percentage,
        'Matched Locations': matched_locations_dict
    }

# Print the findings
for keyword_file, result in findings.items():
    print(f'Findings for {keyword_file}:')
    print(f'Total Keywords: {result["Total Keywords"]}')
    print(f'Locations Matched: {result["Locations Matched"]}')
    print(f'Percentage Match: {result["Percentage Match"]:.2f}%')
    print(f'Matched Locations: {result["Matched Locations"]}\n')


In [ ]:
import pandas as pd
from fuzzywuzzy import process

# Read cleaned locations file
locations_df = pd.read_excel('locations_cleaned.xlsx', header=None, names=['Location'])

# List of other Excel files
keyword_files = ['Keyphrases_aws.xlsx', 'Keyphrases_td-idf.xlsx', 'Keyphrases_yake.xlsx']

# Dictionary to store findings
findings = {}

# Fuzzy matching threshold (adjust as needed)
threshold = 90

# Iterate through each keyword file
for keyword_file in keyword_files:
    # Read keyword file
    keyword_df = pd.read_excel(keyword_file)

    # Convert columns to lowercase for case-insensitive comparison
    keyword_df['Keyword'] = keyword_df['Keyword'].str.lower()
    locations_df['Location'] = locations_df['Location'].str.lower()

    # Dictionary to store matched locations
    matched_locations_dict = {}

    # Iterate through each keyword
    for keyword in keyword_df['Keyword']:
        # Use fuzzywuzzy process to find the best match
        match = process.extractOne(keyword, locations_df['Location'])
        
        # Check if the match meets the threshold
        if match[1] >= threshold:
            # Store the matched location
            matched_locations_dict[keyword] = match[0]

    # Calculate the percentage of locations in each keyword file
    percentage = (len(matched_locations_dict) / len(keyword_df)) * 100

    # Store findings in the dictionary
    findings[keyword_file] = {
        'Total Keywords': len(keyword_df),
        'Locations Matched': len(matched_locations_dict),
        'Percentage Match': percentage,
        'Matched Locations': matched_locations_dict
    }

# Print the findings
for keyword_file, result in findings.items():
    print(f'Findings for {keyword_file}:')
    print(f'Total Keywords: {result["Total Keywords"]}')
    print(f'Locations Matched: {result["Locations Matched"]}')
    print(f'Percentage Match: {result["Percentage Match"]:.2f}%')
    print(f'Matched Locations: {result["Matched Locations"]}\n')

In [ ]:
import pandas as pd
from fuzzywuzzy import process
import re

# Function to clean text
def clean_text(text):
    # Remove spaces, special characters, numbers, commas, points, etc.
    cleaned_text = re.sub(r'[^a-zA-Z]', '', str(text))
    return cleaned_text

# Read cleaned locations file
locations_df = pd.read_excel('locations_cleaned.xlsx', header=None, names=['Location'])
locations_df['Location'] = locations_df['Location'].apply(clean_text)

# List of other Excel files
keyword_files = ['Keyphrases_aws.xlsx', 'Keyphrases_td-idf.xlsx', 'Keyphrases_yake.xlsx']

# Dictionary to store findings
findings = {}

# Fuzzy matching threshold (adjust as needed)
threshold = 92

# Iterate through each keyword file
for keyword_file in keyword_files:
    # Read keyword file
    keyword_df = pd.read_excel(keyword_file)
    
    # Clean the Keyword column in keyword_df
    keyword_df['Keyword'] = keyword_df['Keyword'].apply(clean_text)

    # Convert columns to lowercase for case-insensitive comparison
    keyword_df['Keyword'] = keyword_df['Keyword'].str.lower()
    locations_df['Location'] = locations_df['Location'].str.lower()

    # Dictionary to store matched locations
    matched_locations_dict = {}

    # Iterate through each keyword
    for keyword in keyword_df['Keyword']:
        # Use fuzzywuzzy process to find the best match
        match, score, index = process.extractOne(keyword, locations_df['Location'])
        
        # Check if the match meets the threshold
        if score >= threshold:
            # Store the matched location
            matched_locations_dict[keyword] = match

    # Calculate the percentage of locations in each keyword file
    percentage = (len(matched_locations_dict) / len(keyword_df)) * 100

    # Store findings in the dictionary
    findings[keyword_file] = {
        'Total Keywords': len(keyword_df),
        'Locations Matched': len(matched_locations_dict),
        'Percentage Match': percentage,
        'Matched Locations': matched_locations_dict
    }

# Print the findings
for keyword_file, result in findings.items():
    print(f'Findings for {keyword_file}:')
    print(f'Total Keywords: {result["Total Keywords"]}')
    print(f'Locations Matched: {result["Locations Matched"]}')
    print(f'Percentage Match: {result["Percentage Match"]:.2f}%')
    print(f'Matched Locations: {result["Matched Locations"]}\n')